In [42]:
# !git clone https://github.com/Saiteja-Reddy/Automatic-Text-Scoring.git
import  keras.layers  as  klayers 
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Input, Embedding, GlobalAveragePooling1D, Concatenate, Activation, Lambda, BatchNormalization, Convolution1D, Dropout
from keras.preprocessing.text import Tokenizer
import numpy as np
import nltk
from quadratic_weighted_kappa import QWK
from sklearn.metrics import cohen_kappa_score
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import initializers
from scipy import stats

In [43]:
class Neural_Tensor_layer(Layer):
	def __init__(self,output_dim,input_dim=None, **kwargs):
		self.output_dim=output_dim
		self.input_dim=input_dim
		if self.input_dim:
			kwargs['input_shape']=(self.input_dim,)
# 		print("YAYY", input_dim, output_dim)
		super(Neural_Tensor_layer,self).__init__(**kwargs)

	def call(self,inputs,mask=None):
		e1=inputs[0]
		e2=inputs[1]
		batch_size=K.shape(e1)[0]
		k=self.output_dim
		

		feed_forward=K.dot(K.concatenate([e1,e2]),self.V)

		bilinear_tensor_products = [ K.sum((e2 * K.dot(e1, self.W[0])) + self.b, axis=1) ]

		for i in range(k)[1:]:	
			btp=K.sum((e2*K.dot(e1,self.W[i]))+self.b,axis=1)
			bilinear_tensor_products.append(btp)

		result=K.tanh(K.reshape(K.concatenate(bilinear_tensor_products,axis=0),(batch_size,k))+feed_forward)

		return result
    
	def build(self,input_shape):
		mean=0.0
		std=1.0
		k=self.output_dim
		d=self.input_dim
		##truncnorm generate continuous random numbers in given range
		W_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(k,d,d))
		V_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(2*d,k))
		self.W=K.variable(W_val)
		self.V=K.variable(V_val)
		self.b=K.zeros((self.input_dim,))
		self.trainable_weights=[self.W,self.V,self.b]    

	def compute_output_shape(self, input_shape):
		batch_size=input_shape[0][0]
		return(batch_size,self.output_dim)

In [44]:
class Temporal_Mean_Pooling(Layer): # conversion from (samples,timesteps,features) to (samples,features)
	def __init__(self, **kwargs):
		super(Temporal_Mean_Pooling,self).__init__(**kwargs)
		# masked values in x (number_of_samples,time)
		self.supports_masking=True
		# Specifies number of dimensions to each layer
		self.input_spec=InputSpec(ndim=3)
        
	def call(self,x,mask=None):
		if mask is None:
			mask=K.mean(K.ones_like(x),axis=-1)

		mask=K.cast(mask,K.floatx())
				#dimension size single vec/number of samples
		return K.sum(x,axis=-2)/K.sum(mask,axis=-1,keepdims=True)        

	def compute_mask(self,input,mask):
		return None
    
    
	def compute_output_shape(self,input_shape):
		return (input_shape[0],input_shape[2])

In [45]:
EMBEDDING_DIM=300
MAX_NB_WORDS=4000

MAX_SEQUENCE_LENGTH=500
VALIDATION_SPLIT=0.20
DELTA=20

texts=[]
labels=[]
sentences=[]

originals = []

fp1=open("glove.6B.300d.txt","r", encoding="utf-8")
glove_emb={}
for line in fp1:
	temp=line.split(" ")
	glove_emb[temp[0]]=np.asarray([float(i) for i in temp[1:]])

print("Embedding done")

Embedding done


In [46]:
essay_type = '2'

fp=open("data/training_set_rel3.tsv",'r', encoding="ascii", errors="ignore")
fp.readline()
originals = []
for line in fp:
    temp=line.split("\t")
    if(temp[1]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        originals.append(float(temp[6]))
# print(originals)
fp.close()
# print(originals)
print("range min - ", min(originals) , " ; range max - ", max(originals))

range_min = min(originals)
range_max = max(originals)

# range_min = 1
# range_max = 6

# print("range min - ", range_min , " ; range max - ", range_max)


fp=open("data/training_set_rel3.tsv",'r', encoding="ascii", errors="ignore")
fp.readline()
sentences=[]
for line in fp:
    temp=line.split("\t")
    if(temp[1]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        texts.append(temp[2])
        labels.append((float(temp[6])-range_min)/(range_max-range_min)) ## why ??  - normalize to range [0-1]
        line=temp[2].strip()
        sentences.append(nltk.tokenize.word_tokenize(line))

fp.close()

range min -  1.0  ; range max -  6.0


In [47]:
labels
print(len(labels))

1800


In [48]:
texts[1]

'"Write a persuasive essay to a newspaper reflecting your views on censorship in libraries. Do you believe that certain should be removed i think so be no that yes i think should no person that in chager the book, music, movies, magazines, ect., that be no agure      why do i think if you need that please  think i no thank you please if  i need why do we if know that if i failure the this test i who need to graduate please the children allow to home please yes.          Why do we need to be a prafece person please why do we need to do this why write this assgiment because you mean to be the best teaches ever and ever facebook is my password is @PERSON1  @NUM1 that why i need my myspace is the same thingh but different at same time please know that i need to know i really  i need to my e-mail address is  @EMAIL1 that is my e-mail please work m"'

In [49]:
print("text labels appended %s" %len(texts))

labels=np.asarray(labels)
print(labels)
print(len(labels))

text labels appended 1800
[0.6 0.  0.2 ... 0.2 0.4 0.4]
1800


In [50]:
for i in sentences:
	temp1=np.zeros((1, EMBEDDING_DIM))
	for w in i:
		if(w in glove_emb):
			temp1+=glove_emb[w]
	temp1/=len(i)

In [51]:
tokenizer=Tokenizer(num_words = MAX_NB_WORDS) #num_words=MAX_NB_WORDS) #limits vocabulory size
tokenizer.fit_on_texts(texts) #encoding the text
sequences=tokenizer.texts_to_sequences(texts) #returns list of sequences
word_index=tokenizer.word_index #dictionary mapping, word and specific token for that word...
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH) #padding to max_length


print('Shape of data tensor:', data.shape)

Found 14685 unique tokens.
Shape of data tensor: (1800, 500)


In [52]:
indices=np.arange(data.shape[0]) #with one argument, start=0, step =1
print(data.shape)
np.random.shuffle(indices)
data=data[indices]
print(data.shape)
labels=labels[indices]
# np.reshape(labels, ())
print(labels.shape)
validation_size=int(VALIDATION_SPLIT*data.shape[0])
print(validation_size)

(1800, 500)
(1800, 500)
(1800,)
360


In [53]:
x_train=data[:-validation_size] #data-validation data
print(x_train.shape)
# print(x_train)
# print(labels)
y_train=labels[:-validation_size]
# print(y_train.transpose)
print(y_train.shape)
# y_train = np.reshape(y_train, (1427, 1))
# print(y_train_new)
# print(y_train)
x_val=data[-validation_size:]
print(x_val.shape)
y_val=labels[-validation_size:]

(1440, 500)
(1440,)
(360, 500)


In [54]:
embedding_matrix = np.zeros((len(word_index), EMBEDDING_DIM))
print(embedding_matrix.shape)

(14685, 300)


In [55]:
for word,i in word_index.items():
	if(i>=len(word_index)):
		continue
	if word in glove_emb:
			embedding_matrix[i]=glove_emb[word]
vocab_size=len(word_index)
print(vocab_size)

14685


In [56]:
embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=True,
							trainable=False)
# print(embedding_layer.shape)
side_embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=False,
							trainable=False)

In [57]:
def SKIPFLOW(lstm_dim=50, lr=1e-4, lr_decay=1e-6, k=4, eta=3, delta=50, activation="relu", maxlen=MAX_SEQUENCE_LENGTH, seed=None):
    e = Input(name='essay',shape=(maxlen,))
    print("e", e)
#     trad_feats=Input(shape=(7,))
#     print("trad_feats", trad_feats)
    embed = embedding_layer(e)
    print(embed.shape)
    lstm_layer=LSTM(lstm_dim,return_sequences=True)
    print(lstm_layer)
    hidden_states=lstm_layer(embed)
    htm=Temporal_Mean_Pooling()(hidden_states)    
    side_embed = side_embedding_layer(e)
    side_hidden_states=lstm_layer(side_embed)    
    tensor_layer=Neural_Tensor_layer(output_dim=k,input_dim=lstm_dim)
#     print(input_dim, output_dim)
    pairs = [((eta + i * delta) % maxlen, (eta + i * delta + delta) % maxlen) for i in range(maxlen // delta)]
    hidden_pairs = [ (Lambda(lambda t: t[:, p[0], :])(side_hidden_states), Lambda(lambda t: t[:, p[1], :])(side_hidden_states)) for p in pairs]
    sigmoid = Dense(1, activation="sigmoid", kernel_initializer=initializers.glorot_normal(seed=seed))
    coherence = [sigmoid(tensor_layer([hp[0], hp[1]])) for hp in hidden_pairs]
    co_tm=Concatenate()(coherence[:]+[htm])
    dense = Dense(256, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(co_tm)
    dense = Dense(128, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
    dense = Dense(64, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
    out = Dense(1, activation="sigmoid")(dense)
    model = Model(inputs=[e], outputs=[out])
    print("input", [e])
    print("outputs", out)
    adam = Adam(lr=lr, decay=lr_decay)
    model.compile(loss="mean_squared_error", optimizer=adam, metrics=["MSE"])
    return model

In [58]:
# from keras.utils.vis_utils import plot_model
earlystopping = EarlyStopping(monitor="val_mean_squared_error", patience=5)
sf_2 = SKIPFLOW(lstm_dim=500, lr=2e-4, lr_decay=2e-6, k=4, eta=13, delta=50, activation="relu", seed=None)
sf_2.summary()
# plot_model(sf_1)

e Tensor("essay_1:0", shape=(?, 500), dtype=float32)
(?, 500, 300)
input [<tf.Tensor 'essay_1:0' shape=(?, 500) dtype=float32>]
outputs Tensor("dense_10/Sigmoid:0", shape=(?, 1), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
essay (InputLayer)              (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 500, 300)     4405500     essay[0][0]                      
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 500, 500)     1602000     embedding_3[0][0]                
                                                                 embedding_4[0][0]                
______________

In [59]:
# print(sf_1)
epochs = 100
# epochs = 1000
print(type(x_train))
# y_train = np.asarray(y_train)
print(type(y_train))

sf_2.fit(x_train, y_train, batch_size=32, epochs=epochs, validation_data=([x_val], y_val), callbacks=[earlystopping])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Train on 1440 samples, validate on 360 samples
Epoch 1/100
1440/1440 [==============================] - 87s 61ms/step - loss: 0.0194 - mean_squared_error: 0.0194 - val_loss: 0.0142 - val_mean_squared_error: 0.0142
Epoch 2/100
1440/1440 [==============================] - 81s 56ms/step - loss: 0.0143 - mean_squared_error: 0.0143 - val_loss: 0.0147 - val_mean_squared_error: 0.0147
Epoch 3/100
1440/1440 [==============================] - 83s 58ms/step - loss: 0.0126 - mean_squared_error: 0.0126 - val_loss: 0.0126 - val_mean_squared_error: 0.0126
Epoch 4/100
1440/1440 [==============================] - 83s 57ms/step - loss: 0.0116 - mean_squared_error: 0.0116 - val_loss: 0.0118 - val_mean_squared_error: 0.0118
Epoch 5/100
1440/1440 [==============================] - 82s 57ms/step - loss: 0.0106 - mean_squared_error: 0.0106 - val_loss: 0.0124 - val_mean_squared_error: 0.0124
Epoch 6/100
1440/1440 [==============================] - 82s 57ms/step

In [60]:
y_pred=sf_2.predict([x_val])
y_pred

array([[0.46089828],
       [0.576725  ],
       [0.64377844],
       [0.4184278 ],
       [0.5835984 ],
       [0.42685193],
       [0.41215548],
       [0.12620825],
       [0.6493654 ],
       [0.4587961 ],
       [0.52799934],
       [0.5310778 ],
       [0.5549681 ],
       [0.31933445],
       [0.3062078 ],
       [0.51277786],
       [0.42064697],
       [0.38145876],
       [0.556369  ],
       [0.53551775],
       [0.41446975],
       [0.4121983 ],
       [0.49808177],
       [0.4043721 ],
       [0.37808728],
       [0.32674593],
       [0.48663443],
       [0.4724245 ],
       [0.52522165],
       [0.58121437],
       [0.3770579 ],
       [0.3601788 ],
       [0.5417104 ],
       [0.52304775],
       [0.62351644],
       [0.33476925],
       [0.45366955],
       [0.44276595],
       [0.507017  ],
       [0.50749516],
       [0.52941364],
       [0.5980596 ],
       [0.48773807],
       [0.39015317],
       [0.5122498 ],
       [0.23577136],
       [0.5257977 ],
       [0.384

In [61]:
y_val_fin = [int(round(a*(range_max-range_min)+range_min)) for a in y_val]
print(y_val_fin)

[4, 4, 4, 3, 4, 3, 3, 2, 4, 4, 4, 3, 3, 3, 3, 3, 4, 3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 4, 4, 4, 3, 3, 4, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2, 4, 4, 2, 3, 3, 4, 3, 5, 4, 4, 3, 2, 5, 3, 5, 4, 3, 3, 3, 4, 4, 3, 3, 3, 4, 5, 3, 2, 3, 4, 4, 4, 3, 4, 3, 4, 3, 4, 4, 3, 3, 4, 4, 5, 3, 3, 5, 3, 4, 3, 3, 3, 4, 4, 4, 4, 4, 4, 1, 2, 4, 3, 2, 3, 4, 3, 4, 3, 4, 3, 4, 4, 3, 4, 1, 3, 3, 2, 4, 4, 4, 4, 4, 4, 4, 5, 3, 3, 4, 3, 4, 3, 4, 2, 3, 3, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 5, 5, 2, 3, 4, 4, 4, 3, 3, 4, 4, 4, 3, 4, 3, 3, 2, 3, 4, 4, 4, 4, 5, 2, 4, 3, 4, 4, 3, 3, 3, 3, 2, 5, 4, 4, 4, 3, 3, 3, 3, 1, 4, 3, 2, 3, 3, 3, 3, 4, 3, 3, 3, 2, 3, 4, 3, 3, 4, 3, 1, 3, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 3, 4, 2, 4, 5, 4, 4, 4, 3, 4, 3, 2, 4, 3, 3, 3, 4, 3, 3, 4, 6, 4, 4, 4, 3, 3, 4, 2, 4, 4, 3, 3, 3, 3, 4, 4, 3, 3, 3, 4, 4, 4, 4, 3, 4, 4, 2, 4, 3, 4, 4, 3, 4, 4, 3, 4, 2, 3, 4, 4, 3, 5, 3, 3, 4, 3, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 2, 4, 3, 3, 4, 5, 3, 3, 3, 4, 4, 2, 3, 4, 4, 4, 

In [62]:
y_pred_fin =[int(round(a*(range_max-range_min)+range_min)) for a in y_pred.reshape(360).tolist()]
print(y_pred_fin)

[3, 4, 4, 3, 4, 3, 3, 2, 4, 3, 4, 4, 4, 3, 3, 4, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4, 4, 4, 3, 3, 3, 4, 4, 4, 4, 3, 3, 4, 2, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 4, 3, 4, 4, 4, 3, 3, 3, 3, 4, 4, 3, 3, 4, 4, 4, 3, 2, 3, 3, 3, 3, 4, 4, 2, 3, 3, 3, 4, 3, 3, 4, 4, 4, 3, 3, 4, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 2, 2, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 4, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 4, 2, 3, 4, 3, 4, 3, 3, 2, 2, 2, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 2, 3, 3, 3, 4, 2, 3, 4, 4, 4, 2, 4, 3, 3, 3, 4, 4, 3, 3, 4, 4, 3, 4, 3, 4, 4, 3, 3, 2, 3, 3, 4, 4, 4, 3, 2, 4, 3, 4, 2, 4, 3, 2, 4, 3, 3, 3, 4, 4, 3, 3, 2, 3, 4, 3, 3, 3, 2, 2, 2, 3, 4, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 3, 3, 3, 3, 2, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 4, 4, 3, 3, 3, 2, 4, 3, 3, 3, 3, 2, 4, 4, 3, 3, 4, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 4, 4, 3, 3, 4, 3, 3, 2, 4, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, 4, 4, 2, 4, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 4, 3, 

In [63]:
print(cohen_kappa_score(y_val_fin,y_pred_fin,weights="quadratic"))

0.5930260329591592


In [64]:
def Cmatrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def QWK_new(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = Cmatrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [65]:
QWK_new(y_val_fin, y_pred_fin)

0.5930260329591592

In [66]:
sf_2.save('model_final/2_model.h5')

In [67]:
y_pred*(range_max-range_min)+range_min

array([[3.3044915],
       [3.883625 ],
       [4.218892 ],
       [3.092139 ],
       [3.9179919],
       [3.1342597],
       [3.0607774],
       [1.6310413],
       [4.246827 ],
       [3.2939806],
       [3.6399968],
       [3.655389 ],
       [3.7748406],
       [2.5966723],
       [2.531039 ],
       [3.5638893],
       [3.1032348],
       [2.9072938],
       [3.781845 ],
       [3.6775887],
       [3.0723488],
       [3.0609915],
       [3.490409 ],
       [3.0218606],
       [2.8904364],
       [2.6337297],
       [3.4331722],
       [3.3621225],
       [3.6261082],
       [3.906072 ],
       [2.8852897],
       [2.800894 ],
       [3.708552 ],
       [3.6152387],
       [4.1175823],
       [2.6738462],
       [3.2683477],
       [3.2138298],
       [3.5350852],
       [3.5374758],
       [3.6470683],
       [3.990298 ],
       [3.4386904],
       [2.9507658],
       [3.5612493],
       [2.1788568],
       [3.6289887],
       [2.920365 ],
       [2.6993628],
       [3.4037488],


In [68]:
y_pred_fin

[3,
 4,
 4,
 3,
 4,
 3,
 3,
 2,
 4,
 3,
 4,
 4,
 4,
 3,
 3,
 4,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 3,
 3,
 4,
 2,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 2,
 3,
 3,
 3,
 3,
 4,
 4,
 2,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 4,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 4,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 2,
 3,
 4,
 3,
 4,
 3,
 3,
 2,
 2,
 2,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 2,
 3,
 3,
 3,
 4,
 2,
 3,
 4,
 4,
 4,
 2,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 4,
 4,
 3,
 3,
 2,
 3,
 3,
 4,
 4,
 4,
 3,
 2,
 4,
 3,
 4,
 2,
 4,
 3,
 2,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 2,
 3,
 4,
 3,
 3,
 3,
 2,
 2,
 2,
 3,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 3,
 3,
 3,


In [69]:
y_val_fin

[4,
 4,
 4,
 3,
 4,
 3,
 3,
 2,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 2,
 4,
 4,
 2,
 3,
 3,
 4,
 3,
 5,
 4,
 4,
 3,
 2,
 5,
 3,
 5,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 5,
 3,
 2,
 3,
 4,
 4,
 4,
 3,
 4,
 3,
 4,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 5,
 3,
 3,
 5,
 3,
 4,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 1,
 2,
 4,
 3,
 2,
 3,
 4,
 3,
 4,
 3,
 4,
 3,
 4,
 4,
 3,
 4,
 1,
 3,
 3,
 2,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 3,
 3,
 4,
 3,
 4,
 3,
 4,
 2,
 3,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 5,
 5,
 2,
 3,
 4,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 4,
 3,
 3,
 2,
 3,
 4,
 4,
 4,
 4,
 5,
 2,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 2,
 5,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 1,
 4,
 3,
 2,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 2,
 3,
 4,
 3,
 3,
 4,
 3,
 1,
 3,
 3,
 4,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 4,
 3,
 4,
 2,
 4,
 5,
 4,
 4,
 4,
 3,
 4,


In [70]:
sf_2.save_weights('weights_final/2_weights.h5')

In [ ]:
!pip install pydot
# !pip install graphviz
from keras.utils import plot_model
plot_model(sf_1)

In [3]:
import  keras.layers  as  klayers 
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Input, Embedding, GlobalAveragePooling1D, Concatenate, Activation, Lambda, BatchNormalization, Convolution1D, Dropout
from keras.preprocessing.text import Tokenizer
import numpy as np
import nltk
from quadratic_weighted_kappa import QWK
from sklearn.metrics import cohen_kappa_score
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import initializers
from scipy import stats
import tensorflow as tf

EMBEDDING_DIM=300
MAX_NB_WORDS=4000
MAX_SEQUENCE_LENGTH=500
DELTA=20

fp1=open("glove.6B.300d.txt","r", encoding="utf-8")
glove_emb={}
for line in fp1:
	temp=line.split(" ")
	glove_emb[temp[0]]=np.asarray([float(i) for i in temp[1:]])

print("Embedding done")


class Neural_Tensor_layer(Layer):
	def __init__(self,output_dim,input_dim=None, **kwargs):
		self.output_dim=output_dim
		self.input_dim=input_dim
		if self.input_dim:
			kwargs['input_shape']=(self.input_dim,)
		super(Neural_Tensor_layer,self).__init__(**kwargs)

	def call(self,inputs,mask=None):
		e1=inputs[0]
		e2=inputs[1]
		batch_size=K.shape(e1)[0]
		k=self.output_dim
		

		feed_forward=K.dot(K.concatenate([e1,e2]),self.V)

		bilinear_tensor_products = [ K.sum((e2 * K.dot(e1, self.W[0])) + self.b, axis=1) ]

		for i in range(k)[1:]:	
			btp=K.sum((e2*K.dot(e1,self.W[i]))+self.b,axis=1)
			bilinear_tensor_products.append(btp)

		result=K.tanh(K.reshape(K.concatenate(bilinear_tensor_products,axis=0),(batch_size,k))+feed_forward)

		return result
    
	def build(self,input_shape):
		mean=0.0
		std=1.0
		k=self.output_dim
		d=self.input_dim
		W_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(k,d,d))
		V_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(2*d,k))
		self.W=K.variable(W_val)
		self.V=K.variable(V_val)
		self.b=K.zeros((self.input_dim,))
		self.trainable_weights=[self.W,self.V,self.b]    

	def compute_output_shape(self, input_shape):
		batch_size=input_shape[0][0]
		return(batch_size,self.output_dim)

class Temporal_Mean_Pooling(Layer):
	def __init__(self, **kwargs):
		super(Temporal_Mean_Pooling,self).__init__(**kwargs)
		self.supports_masking=True
		self.input_spec=InputSpec(ndim=3)
        
	def call(self,x,mask=None):
		if mask is None:
			mask=K.mean(K.ones_like(x),axis=-1)

		mask=K.cast(mask,K.floatx())
		return K.sum(x,axis=-2)/K.sum(mask,axis=-1,keepdims=True)        

	def compute_mask(self,input,mask):
		return None
    
    
	def compute_output_shape(self,input_shape):
		return (input_shape[0],input_shape[2])

def SKIPFLOW(embedding_matrix, vocab_size, lstm_dim=50, lr=1e-4, lr_decay=1e-6, k=5, eta=3, delta=50, activation="relu", maxlen=MAX_SEQUENCE_LENGTH, seed=None):
                                
	embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
								input_length=MAX_SEQUENCE_LENGTH,
								mask_zero=True,
								trainable=False)
	side_embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
								input_length=MAX_SEQUENCE_LENGTH,
								mask_zero=False,
								trainable=False)
	e = Input(name='essay',shape=(maxlen,))
	# trad_feats=Input(shape=(7,))
	embed = embedding_layer(e)
	lstm_layer=LSTM(lstm_dim,return_sequences=True)
	hidden_states=lstm_layer(embed)
	htm=Temporal_Mean_Pooling()(hidden_states)
	side_embed = side_embedding_layer(e)
	side_hidden_states=lstm_layer(side_embed)	
	# lstm_layer=LSTM(lstm_dim,return_sequences=True)
	tensor_layer=Neural_Tensor_layer(output_dim=k,input_dim=lstm_dim)
	pairs = [((eta + i * delta) % maxlen, (eta + i * delta + delta) % maxlen) for i in range(maxlen // delta)]
	hidden_pairs = [ (Lambda(lambda t: t[:, p[0], :])(side_hidden_states), Lambda(lambda t: t[:, p[1], :])(side_hidden_states)) for p in pairs]
	sigmoid = Dense(1, activation="sigmoid", kernel_initializer=initializers.glorot_normal(seed=seed))
	coherence = [sigmoid(tensor_layer([hp[0], hp[1]])) for hp in hidden_pairs]
	co_tm=Concatenate()(coherence[:]+[htm])
	dense = Dense(256, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(co_tm)
	dense = Dense(128, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
	dense = Dense(64, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
	out = Dense(1, activation="sigmoid")(dense)
	model = Model(inputs=[e], outputs=[out])
	adam = Adam(lr=lr, decay=lr_decay)
	model.compile(loss="mean_squared_error", optimizer=adam, metrics=["MSE"])
	return model


def init(essay_type = '4'): 

	EMBEDDING_DIM=300
	MAX_NB_WORDS=4000
	MAX_SEQUENCE_LENGTH=500
	DELTA=20

	texts=[]
	originals = []

	fp=open("C:/Users/mehar/Desktop/MIDAS_CallingBluff/skipflow/data/training_set_rel3.tsv",'r', encoding="ascii", errors="ignore")
	fp.readline()
	sentences=[]
	doctovec=[]
	for line in fp:
	    temp=line.split("\t")
	    if(temp[1]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
	        texts.append(temp[2])
	        originals.append(float(temp[6]))
	        line=temp[2].strip()
	fp.close()

	range_min = min(originals)
	range_max = max(originals)
	print("range min - ", min(originals) , " ; range max - ", max(originals))

	tokenizer=Tokenizer() #num_words=MAX_NB_WORDS) #limits vocabulory size
	tokenizer.fit_on_texts(texts)
	sequences=tokenizer.texts_to_sequences(texts) 
	word_index=tokenizer.word_index #dictionary mapping

	# print('Found %s unique tokens.' % len(word_index)) 
	vocab_size = len(word_index)
	embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

	for word,i in word_index.items():
		if(i>=len(word_index)):
			continue
		if word in glove_emb:
				embedding_matrix[i]=glove_emb[word]     	

	earlystopping = EarlyStopping(monitor="val_mean_squared_error", patience=5)
	sf_1 = SKIPFLOW(embedding_matrix, vocab_size ,lstm_dim=50, lr=2e-4, lr_decay=2e-6, k=4, eta=13, delta=50, activation="relu", seed=None)

	sf_1.load_weights("C:/Users/mehar/Desktop/MIDAS_CallingBluff/skipflow/Automatic-Text-Scoring/webapp/models/weights_final/" + str(essay_type) + "_weights.h5")
	print("Loaded Model from disk")


	graph = tf.get_default_graph()

	print("Essay type done loading ::: ", essay_type)

	return sf_1,graph,glove_emb, tokenizer, range_min, range_max


Embedding done
